In [ ]:
# default_exp convert_files

# convert_files

> Converts existing repositories to proper csv with headers


In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
from nbdev import *

In [ ]:
#export
#! python
import re
import csv
from os import listdir


def read__write_from_sep_sv(filename,out_filename,in_sep=None,out_sep = None,encoding=None,chop_end_field=True):
    """Reads in a sep file and outputs in proper csv"""
    if in_sep is None:
        in_sep = '|'
    if out_sep is None:
        out_sep = ','
    
    # TODO: see if we can fix or handle encoding errors? Hack is to set to latin-1
    if encoding is None:
        encoding="latin-1"
    line_number = 0
   
    fp = open(filename, "r", encoding=encoding)
    
    row_list =[]
    inconsistent_error = False
    with open(filename, 'r', encoding=encoding) as fp:
        for line in fp:
            line_number += 1
            line = line.strip()

            fields = line.split(in_sep)  # TODO: use csv reader ?     
            if chop_end_field:
                fields.pop()
            if line_number == 1:
                number_of_fields = len(fields)
            else:
                if len(fields) != number_of_fields:
                    # since some fields originaly contained new lines 
                    # and now the lines have been sorted, the data is kind of scrambled
                    # so this is a fudge padding out the missing fields.
                    no_missing_fields = number_of_fields - len(fields)
                    for i in range(no_missing_fields):
                        fields.append('')
            row_list.append(fields)
        
    with open(out_filename,'w',encoding=encoding) as fo:
        writer = csv.writer(fo,quoting=csv.QUOTE_MINIMAL,delimiter=out_sep)
        writer.writerows(row_list)

def write_from_sep_sv(input_list,out_filename,in_sep=None,out_sep = None,encoding=None,chop_end_field=True):
    """Reads in a sep file and outputs in proper csv"""
    if in_sep is None:
        in_sep = '|'
    if out_sep is None:
        out_sep = ','
    
    # TODO: see if we can fix or handle encoding errors? Hack is to set to latin-1
    if encoding is None:
        encoding="latin-1"
    line_number = 0
   
    row_list = []
    inconsistent_error = False
    for line in input_list:
        line_number += 1
        line = line.strip()

        fields = line.split(in_sep)  # TODO: use csv reader ?     
        if chop_end_field:
            fields.pop()
        if line_number == 1:
            header = line
            number_of_fields = len(fields)
        else:
            if len(fields) != number_of_fields:
                # since some fields originaly contained new lines 
                # and now the lines have been sorted, the data is kind of scrambled
                # so this is a fudge padding out the missing fields.
                no_missing_fields = number_of_fields - len(fields)
                for i in range(no_missing_fields):
                    fields.append('')
        row_list.append(fields)
        
    with open(out_filename,'w',encoding=encoding) as fo:
        writer = csv.writer(fo,quoting=csv.QUOTE_MINIMAL,delimiter=out_sep)
        writer.writerows(row_list)

def read_headers(header_file):
    """Reads the header file which consists of lines starting with the table name followed by a : and the header.
    It returns a dictionary of table names and their associated header lines.
    """
    header_lines = {}
    with open(header_file, 'r') as f:
        for line in f:
            table_name, header = parse_header_line(line.rstrip("\n"))
            if table_name is not None:
                header_lines[table_name] = header



    return header_lines

def parse_header_line(line):
    m = re.match('^([^:]+):', line)
    if m:
        table_name = m.group(1)
        line = line[m.end():]
        return table_name,line
    return None,None

def restore_header(filename,output_filename,header,encoding ="latin-1"): 
    with open(filename, 'r',encoding=encoding) as f:
        with open(output_filename, 'w',encoding=encoding) as out:
            out.write(header+'\n')
            for line in f:
                line = line.rstrip('\n')
                if line == header:
                    continue
                out.write(line+'\n')

def restore_header_in_mem(filename,header,encoding ="latin-1"): 
    
    raw_list = []
    header_part = header[0:20]
    header_match = header # if nothing in the file matches then we just use the default one
    with open(filename, 'r',encoding=encoding) as f:
        no_matches = 0
        line_no = 0
        for line in f:
            line = line.rstrip('\n')
            raw_list.append(line)
            line_no += 1
            if line.startswith(header_part):
                header_match = line
                no_matches += 1
            if no_matches > 1:
                print(f"{filename} matched header more than once line: {line_no}",file=sys.stderr)
            
    row_list = []
    with open(filename, 'r',encoding=encoding) as f:
        row_list.append(header_match)
        for line in f:
            line = line.rstrip('\n')
            if line == header_match:
                continue
            row_list.append(line)
    return row_list


        
def convert_directory(src_dir,dest_dir,subdir,headers):
    
    src_path = src_dir + '/' + subdir
    dest_path = dest_dir+'/'+ subdir
   
    if not os.path.isdir(dest_path):
        os.mkdir(dest_path)   
        
    
    #print(src_dir,dest_dir,subdir,dest_path)
        
    files = [f for f in listdir(src_path)]
    for filename in files:
        if not os.path.isfile(src_path+'/'+filename):
            continue
        base=os.path.basename(filename)
        table_name = os.path.splitext(base)[0]
        output_filename = dest_path +'/' + table_name + '.csv'
        if table_name in headers:
            row_list = restore_header_in_mem(src_path+'/'+filename,headers[table_name])
            write_from_sep_sv(row_list,output_filename,in_sep=None,out_sep = None,encoding=None,chop_end_field=True)
        else:
            print(f"Skipping file:{filename}  - no associated header found", file=sys.stderr)       
                
def convert_dirs(exportsdir, dest, server,headers):
    """
    Copy the files into the repository
    """

    path = exportsdir + '/' + server

    dest_path = dest + '/' + server

    if not os.path.isdir(dest_path):
        os.mkdir(dest_path)

    dirs = [f for f in listdir(path)]
    dirs.sort()
    for subdir_name in dirs:
        if os.path.isdir(path+'/'+subdir_name):
            convert_directory(path,dest_path, subdir_name,headers)

In [ ]:
#export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False
    
if __name__ == "__main__" and not IN_NOTEBOOK:
    import argparse
    import os
    import sys

    ap = argparse.ArgumentParser()
    ap.add_argument("-s", "--server", required=True, help="name of the server")
    ap.add_argument("-e", "--exportsdir", required=False,
                help="directory of the current config to be converted")
    ap.add_argument("-d", "--dest", required=True, help="destination directory")
    ap.add_argument("-n", "--header", required=True, help="headers file for restore")
    
    server = args["server"]
    exportsdir = args["exportsdir"]
    dest_directory = args["dest"]

    headers = read_headers(headerfile)

    
    
    convert_dirs(exportsdir,dest_directory,server,headers)
    
    

In [ ]:
#hide
current_extracts_dir = '/Users/bt/localdev/CONFIG_TABLE_EXTRACTS/exports'
dest_dir = 'test'
server_name = 'server2'
headers = read_headers('scripts/headers.txt')
convert_dirs(current_extracts_dir,dest_dir,server_name,headers)

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

In [ ]:
server_name = 'server1'
headers = read_headers('scripts/headers.txt')
convert_dirs(current_extracts_dir,dest_dir,server_name,headers)

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f

Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping file:ATTRIBUTE_SUBSETS.txt  - no associated header found
Skipping f